In [1]:
#%%
import numpy as np
import pandas as pd
import pylab as plt
from time import sleep
from IPython import display

In [2]:
### Fetch the data and load it in pandas
data = pd.read_csv('train.csv')
print "Size of the data: ", data.shape

Size of the data:  (15120, 56)


In [3]:
#%%
# See data (five rows) using pandas tools
print data.head()

   Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0   1       2596      51      3                               258   
1   2       2590      56      2                               212   
2   3       2804     139      9                               268   
3   4       2785     155     18                               242   
4   5       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm     ...      Soil_Type32  \
0            221             232            148     ...                0   
1           

In [13]:
### Prepare input to scikit and train and test cut

binary_data = data[np.logical_or(data['Cover_Type'] == 1,data['Cover_Type'] == 2)] # two-class classification set
X = binary_data.drop('Cover_Type', axis=1).values
y = binary_data['Cover_Type'].values
print np.unique(y)
y = 2 * y - 3 # converting labels from [1,2] to [-1,1]

[1 2]


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
14700,14701,3156,348,19,277,42,499,180,204,160,...,0,0,0,0,0,0,0,0,0,1
13997,13998,3343,128,17,787,104,2916,246,227,105,...,0,1,0,0,0,0,0,0,0,1
7858,7859,3163,103,5,1260,294,2962,229,233,138,...,1,0,0,0,0,0,0,0,0,2
14637,14638,3071,28,10,85,5,1426,216,218,137,...,1,0,0,0,0,0,0,0,0,1
8913,8914,2961,106,24,124,35,1621,252,204,65,...,0,0,0,0,0,0,0,0,0,2
664,665,3011,73,9,659,7,4341,231,223,124,...,0,0,0,0,0,0,0,0,0,2
14730,14731,3210,123,7,743,0,1019,231,235,137,...,0,1,0,0,0,0,0,0,0,2
14856,14857,2613,161,16,30,-10,3284,234,242,134,...,0,0,0,0,0,0,0,0,0,1
1828,1829,3309,268,7,618,28,3711,202,243,179,...,0,0,0,0,0,0,0,1,0,1
10093,10094,2794,60,10,30,11,1940,228,218,122,...,0,0,0,0,0,0,0,0,0,1


In [9]:
#%%
# Import cross validation tools from scikit
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=None)

C:\Users\Louis\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
#%%
### Train a single decision tree

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=8)

# Train the classifier and print training time
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [14]:
#%%
# Do classification on the test dataset and print classification results
from sklearn.metrics import classification_report
target_names = data['Cover_Type'].unique().astype(str).sort()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

         -1       0.72      0.76      0.74       445
          1       0.73      0.68      0.71       419

avg / total       0.72      0.72      0.72       864



In [ ]:
#%%
# Compute accuracy of the classifier (correctly classified instances)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [51]:
#===================================================================
#%%
### Train AdaBoost

# Your first exercise is to program AdaBoost.
# You can call *DecisionTreeClassifier* as above, 
# but you have to figure out how to pass the weight vector (for weighted classification) 
# to the *fit* function using the help pages of scikit-learn. At the end of 
# the loop, compute the training and test errors so the last section of the code can 
# plot the learning curves. 
# 
# Once the code is finished, play around with the hyperparameters (D and T), 
# and try to understand what is happening.

D = 2 # tree depth
T = 1000 # number of trees
w = np.ones(X_train.shape[0]) / X_train.shape[0]
training_scores = np.zeros(X_train.shape[0])
test_scores = np.zeros(X_test.shape[0])

ts = plt.arange(len(training_scores))
training_errors = []
test_errors = []

#===============================
alpha_l=list()
y=np.zeros(X_train.shape[0])
for t in range(500):
    clf.fit(X_train,y_train,sample_weight=w)
    yt=clf.predict(X_train)
    gamma=np.dot(w,yt!=y_train)/sum(w)
    alpha=np.log((1-gamma)/gamma)
    
    w=w*np.exp(alpha*yt!=y_train)
    #alpha_l.append(alpha)
    y+=alpha*yt
    
    
    # Your code should go here
    

In [52]:
y

array([-1.2297335 , -1.2297335 , -1.2297335 , ...,  1.23840033,
        1.23840033,  1.23840033])

In [28]:
alpha_l

[1.7332400480327601,
 1.7332400480328181,
 1.7332400480327887,
 1.7332400480327315,
 1.7332400480327315,
 1.7332400480327315,
 1.7332400480327315,
 1.7332400480327315,
 1.7332400480327101,
 1.7332400480326617,
 1.7332400480327728,
 1.7332400480327679,
 1.7332400480326284,
 1.733240048032697,
 1.7332400480327435,
 1.7332400480327572,
 1.7332400480327848,
 1.7332400480327419,
 1.7332400480328283,
 1.7332400480327268,
 1.7332400480326728,
 1.7332400480327446,
 1.7332400480327612,
 1.7332400480327332,
 1.7332400480326808,
 1.733240048032757,
 1.7332400480327563,
 1.7332400480327426,
 1.7332400480327372,
 1.7332400480328038,
 1.7332400480328167,
 1.7332400480327945,
 1.7332400480327572,
 1.733240048032809,
 1.733240048032811,
 1.7332400480327956,
 1.7332400480327175,
 1.7332400480327652,
 1.7332400480328274,
 1.7332400480328136,
 1.7332400480326506,
 1.733240048032723,
 1.7332400480327748,
 1.7332400480328205,
 1.7332400480327597,
 1.7332400480327801,
 1.7332400480327548,
 1.733240048032635

In [17]:
(clf.predict(X_train)!=y_train)+np.zeros(X_train.shape[0])

array([ 0.,  0.,  1., ...,  0.,  0.,  0.])

In [ ]:
#===============================

#  Plot training and test error    
plt.plot(training_errors, label="training error")
plt.plot(test_errors, label="test error")
plt.legend()



#===================================================================
#%%
### Optional part
### Optimize AdaBoost

# Your final exercise is to optimize the tree depth in AdaBoost. 
# Copy-paste your AdaBoost code into a function, and call it with different tree depths 
# and, for simplicity, with T = 100 iterations (number of trees). Plot the final 
# test error vs the tree depth. Discuss the plot.

#===============================

# Your code should go here
    

#===============================